In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import sys
import os

sys.path.append(os.path.join(os.getcwd(), '/home/jabez/Documents/week_9/Crypto-Trading-Engineering/scripts'))
import index_fund

In [35]:
df_btc = pd.read_csv('/home/jabez/Documents/week_9/Crypto-Trading-Engineering/data/BTC-USD.csv')
df_eth= pd.read_csv('/home/jabez/Documents/week_9/Crypto-Trading-Engineering/data/ETH-USD.csv')
df_bnb= pd.read_csv('/home/jabez/Documents/week_9/Crypto-Trading-Engineering/data/BNB-USD (1).csv')
df_sol= pd.read_csv('/home/jabez/Documents/week_9/Crypto-Trading-Engineering/data/SOL-USD.csv')
df_xrp= pd.read_csv('/home/jabez/Documents/week_9/Crypto-Trading-Engineering/data/XRP-USD.csv')

In [36]:
df_btc = df_btc[['Date', 'Close']].rename(columns={'Close': 'btc_price'})
df_eth = df_eth[['Date', 'Close']].rename(columns={'Close': 'eth_price'})
df_bnb = df_bnb[['Date', 'Close']].rename(columns={'Close': 'bnb_price'})
df_sol = df_sol[['Date', 'Close']].rename(columns={'Close': 'sol_price'})
df_xrp = df_xrp[['Date', 'Close']].rename(columns={'Close': 'xrp_price'})

# Merge data frames on 'Date' column
df = df_btc.merge(df_eth, on='Date').merge(df_bnb, on='Date').merge(df_sol, on='Date').merge(df_xrp, on='Date')


In [37]:
# Convert the 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

# Convert all other columns to numeric, coerce errors to NaN
df = df.apply(pd.to_numeric, errors='coerce')

In [38]:
df

,btc_price,eth_price,bnb_price,sol_price,xrp_price
Date,,,,,
2023-06-20,28327.488281,1792.119995,247.674438,16.635231,0.492870
2023-06-21,30027.296875,1891.007202,249.286346,17.257046,0.500695
2023-06-22,29912.281250,1872.942993,240.792435,16.642620,0.494975
2023-06-23,30695.468750,1892.862061,244.564941,17.149534,0.496566
2023-06-24,30548.695313,1876.059692,236.659653,16.652863,0.487313
...,...,...,...,...,...
2024-06-16,66639.046875,3620.563477,610.054016,151.125504,0.488832
2024-06-17,66490.296875,3511.378906,604.467346,143.334274,0.505160
2024-06-18,65140.746094,3483.681396,588.702087,137.503998,0.490948


In [39]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
mu = expected_returns.mean_historical_return(df)
sigma = risk_models.sample_cov(df)

In [40]:
# Create an Efficient Frontier object
ef_low_risk = EfficientFrontier(mu, sigma)

# Optimize for the minimum volatility
weights_low_risk = ef_low_risk.min_volatility()

# Clean the weights
cleaned_weights_low_risk = ef_low_risk.clean_weights()

# Display the optimized weights
print("Weights for Lowest Risk Portfolio:")
print(cleaned_weights_low_risk)

# Portfolio performance
performance_low_risk = ef_low_risk.portfolio_performance(verbose=True)


Weights for Lowest Risk Portfolio:
OrderedDict([('btc_price', 0.55541), ('eth_price', 0.06774), ('bnb_price', 0.36069), ('sol_price', 0.0), ('xrp_price', 0.01616)])
Expected annual return: 75.9%
Annual volatility: 36.2%
Sharpe Ratio: 2.04


#### Breakdown of Portfolio Weights
BTC: 55.54%
ETH: 6.77%
BNB: 36.07%
SOL: 0.00%
XRP: 1.62%

#### Interpretation of Results
- Expected Annual Return: 75.9% – This is a high expected return, indicating that the portfolio has significant growth potential.
- Annual Volatility: 36.2% – This represents the portfolio's risk, showing how much the returns can deviate from the expected return. A volatility of 36.2% is - - relatively high, reflecting the inherent risk of investing in volatile cryptocurrencies.
- Sharpe Ratio: 2.04 – A Sharpe ratio of 2.04 is very good, indicating that the portfolio offers a high return for each unit of risk taken. Generally, a Sharpe - ratio above 1 is considered good, above 2 is very good, and above 3 is excellent.

In [41]:
# Create an Efficient Frontier object
ef_high_sharpe = EfficientFrontier(mu, sigma)

# Optimize for the maximal Sharpe ratio
weights_high_sharpe = ef_high_sharpe.max_sharpe()

# Clean the weights
cleaned_weights_high_sharpe = ef_high_sharpe.clean_weights()

# Display the optimized weights
print("Weights for Highest Sharpe Ratio Portfolio:")
print(cleaned_weights_high_sharpe)

# Portfolio performance
performance_high_sharpe = ef_high_sharpe.portfolio_performance(verbose=True)


Weights for Highest Sharpe Ratio Portfolio:
OrderedDict([('btc_price', 0.0), ('eth_price', 0.0), ('bnb_price', 0.02788), ('sol_price', 0.97212), ('xrp_price', 0.0)])
Expected annual return: 312.8%
Annual volatility: 74.4%
Sharpe Ratio: 4.18


#### Interpretation of Results
- Expected Annual Return: 312.8% – This is an extremely high expected return, indicating significant growth potential for this portfolio.
- Annual Volatility: 74.4% – This high volatility represents the significant risk associated with this portfolio. It means the portfolio's returns can deviate substantially from the expected return.
- Sharpe Ratio: 4.18 – A Sharpe ratio of 4.18 is exceptionally high, suggesting that the portfolio provides a very high return per unit of risk taken.
Practical Use of Results
- Portfolio Allocation: The portfolio is almost entirely allocated to Solana (SOL), with a small fraction in Binance Coin (BNB). This suggests that, according to the optimization process, Solana provides the best risk-adjusted return within the context of the available data.
- Investment Decision: While this portfolio offers high potential returns, it is also very risky due to its concentration in a single asset. Investors should carefully consider their risk tolerance and investment horizon before adopting such a strategy.